In [322]:
import pandas as pd
import numpy as np
import math

from tqdm import tqdm
import time

import matplotlib.pyplot as plt

import torch
import torch.nn as nn
from torch.autograd import Variable

import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import confusion_matrix

In [323]:
RANDOM_SEED = 13022022

In [324]:
new_df = pd.read_csv('new_df.csv')

In [325]:
new_df.sample()

,Active,train,len,4,5,6,7,8,9,10,...,701,702,703,704,705,706,707,708,709,710
1840,0,1,40,45,45,35,42,45,45,45,...,0,0,0,0,0,0,0,0,0,0


In [326]:
train_new = new_df[new_df['train'] == 1]
test_new =new_df[new_df['train'] == 0]

In [327]:
X = train_new.drop(['Active', 'train'], axis = 1)
y = train_new['Active']

In [328]:
X_test = test_new.drop(['Active', 'train'], axis = 1)

In [329]:
X.sample()

,len,4,5,6,7,8,9,10,11,12,...,701,702,703,704,705,706,707,708,709,710
2370,53,45,43,42,43,43,43,41,43,36,...,0,0,0,0,0,0,0,0,0,0


In [330]:
features = np.array(X)

In [331]:
test_features = np.array(X_test)

In [332]:
x_test = torch.tensor(test_features)
x_test = x_test.type(torch.FloatTensor)
x_test.size()

torch.Size([1614, 708])

In [333]:
x_train = torch.tensor(features)
x_train = x_train.type(torch.FloatTensor)
x_train.size()

torch.Size([5557, 708])

In [334]:
x_labels = torch.tensor(y)
x_labels = x_labels.type(torch.FloatTensor)
x_labels.size()

torch.Size([5557])

In [335]:
train_labels

array([[0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [336]:
nX = 708
nH1 = 500
nH2 = 350
nH3 = 150
nH4 = 75
nH5 = 30
nH6 = 15
nH7 = 7
nY = 1
 
class TwoLayersNet(nn.Module):
    def __init__(self, nX, nH1, nH2, nH3, nH4, nH5, nH6, nH7, nY):        
        super(TwoLayersNet, self).__init__()     # конструктор предка с этим именем
         
        self.fc1 = nn.Linear(nX, nH1)             # создаём параметры модели
        self.fc2 = nn.Linear(nH1, nH2)             # создаём параметры модели
        self.fc3 = nn.Linear(nH2, nH3)             # создаём параметры модели
        self.fc4 = nn.Linear(nH3, nH4)             # создаём параметры модели
        self.fc5 = nn.Linear(nH4, nH5)             # создаём параметры модели
        self.fc6 = nn.Linear(nH5, nH6)             # создаём параметры модели
        self.fc7 = nn.Linear(nH6, nH7)             # создаём параметры модели
        self.fc8 = nn.Linear(nH7, nY)             # в полносвязных слоях
          
    def forward(self, x):                        # задаётся прямой проход
        x = self.fc1(x)                          # выход первого слоя
        x = nn.Sigmoid()(x)                      # пропускаем через Sigmoid
        x = self.fc2(x)                          # выход второго слоя
        x = nn.Sigmoid()(x)                      # пропускаем через сигмоид 
        x = self.fc3(x)                          # выход второго слоя
        x = nn.Sigmoid()(x)                      # пропускаем через сигмоид 
        x = self.fc4(x)                          # выход второго слоя
        x = nn.Sigmoid()(x)                      # пропускаем через сигмоид 
        x = self.fc5(x)                          # выход второго слоя
        x = nn.Sigmoid()(x)                      # пропускаем через сигмоид 
        x = self.fc6(x)                          # выход второго слоя
        x = nn.Sigmoid()(x)                      # пропускаем через сигмоид 
        x = self.fc7(x)                          # выход второго слоя
        x = nn.Sigmoid()(x)                      # пропускаем через сигмоид 
        x = self.fc8(x)                          # выход второго слоя
        x = nn.Sigmoid()(x)                      # пропускаем через сигмоид 
        return x
          
model = TwoLayersNet(nX, nH1, nH2, nH3, nH4, nH5, nH6, nH7, nY)

In [337]:
model = TwoLayersNet(nX, nH1, nH2, nH3, nH4, nH5, nH6, nH7, nY)                           # экземпляр сети        
 
loss      = nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(),          # параметры модели
                            lr=0.5, momentum=0.8) 

In [338]:
def fit(model, x_train,x_labels, batch_size=100, train=True):    
    model.train(train)                                 # важно для Dropout, BatchNorm
    sumL, sumA, numB = 0, 0, int( len(x_train)/batch_size )  # ошибка, точность, батчей
       
    for i in range(0, numB*batch_size, batch_size):          
        xb = x_train[i: i+batch_size]                          # текущий батч,
        yb = x_labels[i: i+batch_size]                          # X,Y - torch тензоры
              
        y = model(xb)[:,0]                                    # прямое распространение

        L = loss(y, yb)                                  # вычисляем ошибку
  
        if train:                                        # в режиме обучения
            optimizer.zero_grad()                        # обнуляем градиенты        
            L.backward()                                 # вычисляем градиенты            
            optimizer.step()                             # подправляем параметры
                                     
        sumL += L.item()                                 # суммарная ошибка (item из графа)
        sumA += (y.round() == yb).float().mean()         # точность определения класса
         
    return sumL/numB,  sumA/numB 

In [339]:
print( "before:      loss: %.4f accuracy: %.4f" %  fit(model, x_train,x_labels, train=False) )
 
epochs = 1000                                            # число эпох
for epoch in tqdm(range(epochs)):                              # эпоха - проход по всем примерам
    L,A = fit(model, x_train, x_labels)                               # одна эпоха
     
    if epoch % 500 == 0 or epoch == epochs-1:                 
        print(f'epoch: {epoch:5d} loss: {L:.4f} accuracy: {A:.4f}' ) 

  0%|                                                                                         | 0/1000 [00:00<?, ?it/s]

before:      loss: 0.5462 accuracy: 0.9629


  0%|                                                                                 | 1/1000 [00:01<17:48,  1.07s/it]

epoch:     0 loss: 0.1745 accuracy: 0.9629


 50%|███████████████████████████████████████▌                                       | 501/1000 [08:56<08:25,  1.01s/it]

epoch:   500 loss: 0.1588 accuracy: 0.9629


100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [17:28<00:00,  1.05s/it]

epoch:   999 loss: 0.1588 accuracy: 0.9629


In [346]:
y_pred = model(x_test)

In [347]:
y_pred = y_pred[:,0]

In [348]:
y_pred

tensor([0.0409, 0.0409, 0.0409,  ..., 0.0409, 0.0409, 0.0409],
       grad_fn=<SelectBackward0>)

In [349]:
test = pd.read_csv('Task/test.csv')
test.drop(['Unnamed: 0'], axis = 1, inplace = True)

In [364]:
test['Active'] = y_pred.detach().numpy()

In [365]:
test.sample()

,Smiles,Active
716,Cl.Cl.N=C(NCCCCCCNC(=N)NC(=N)Nc1ccc(Cl)cc1)NC(...,0.040947


In [366]:
test['Active'] = test['Active'].apply(lambda x: x*1000000-40947)

In [362]:
test['Active'] = test['Active'].apply(lambda x: int(abs(round(x, 0))))

In [367]:
test['Active'].value_counts()

0.221220    270
0.191417    234
0.210044    186
0.202593    182
0.180241    163
0.228670    136
0.239846     69
0.292000     46
0.172791     45
0.314352     44
0.321802     43
0.303176     42
0.247297     40
0.284549     35
0.265923     27
0.258472     26
0.273374     16
0.332978      8
0.161615      2
Name: Active, dtype: int64

In [358]:
test.sample(5)

,Smiles,Active
68,O=C(NOCCO)c1cc(CN2OCCCC2=O)c(F)c(F)c1Nc1ccc(I)...,0.040947
915,CC(O)(CC(=O)O)CC(=O)O,0.040947
1442,CC(C)(C)NC[C@H](O)c1ccc(O)c(CO)c1.Cl,0.040947
1418,CCOc1cc(N)c(Cl)cc1C(=O)NCC1CN(Cc2ccc(F)cc2)CCO...,0.040947
702,NC(=O)C1=CN([C@@H]2O[C@H](CO[P@](=O)(O)O[P@@](...,0.040947


In [320]:
test.to_csv('test2.csv', index=False)